In [5]:

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split


In [75]:
import os
import numpy as np
# Constants
IMAGE_SIZE = (128, 128)
DATASET_PATH = r"D:\CNN Project\mvtec_anomaly_detection"
PROCESSED_PATH = "processed_data"
os.makedirs(PROCESSED_PATH, exist_ok=True)




In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Constants
IMAGE_SIZE = (128, 128)
DATASET_PATH = r"D:\CNN Project\mvtec_anomaly_detection\bottle"
PROCESSED_PATH = "processed_data"
os.makedirs(PROCESSED_PATH, exist_ok=True)

def is_image_file(filename):
    """Check if the file is a valid image format."""
    return filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))

# Define Image Size
IMAGE_SIZE = (128, 128)

def preprocess_image(image):
    """Load and preprocess image for model input."""
    
    if isinstance(image, str):  # If it's a file path (training mode)
        if not os.path.isfile(image):
            print(f"Skipping non-file: {image}")
            return None
        
        image = cv2.imread(image, cv2.IMREAD_COLOR)
        if image is None:
            print(f"Failed to read image: {image}")
            return None

    elif isinstance(image, Image.Image):  # If it's a PIL image (Streamlit mode)
        image = image.convert("RGB")  # Ensure RGB format
        image = np.array(image)  # Convert PIL to NumPy (H, W, C)

    else:
        print("Invalid input type")
        return None
    
    # Resize and Normalize
    image = cv2.resize(image, IMAGE_SIZE)
    image = image.astype(np.float32) / 255.0  # Normalize to [0,1]
    image = np.expand_dims(image, axis=0)  # Add batch dimension for model
    return image

def load_images():
    """Load 'good' and 'anomaly' images with labels."""
    data, labels = [], []
    
    
    

    # Load GOOD images (Label: 0)
    good_path = os.path.join(DATASET_PATH, "train", "good")
    if os.path.exists(good_path) and os.path.isdir(good_path):
        for img_name in os.listdir(good_path):
            img_path = os.path.join(good_path, img_name)
            if is_image_file(img_name):
                image = preprocess_image(img_path)
                if image is not None:
                    data.append(image)
                    labels.append(0)  # Good images labeled as 0
    else:
        print(f"Path not found: {good_path}")

    # Load ANOMALY images (Label: 1) from all subfolders except "good"
    test_path = os.path.join(DATASET_PATH, "test")
    if os.path.exists(test_path) and os.path.isdir(test_path):
        for subfolder in os.listdir(test_path):
            subfolder_path = os.path.join(test_path, subfolder)

            # Skip the "good" subfolder in test data
            if subfolder.lower() == "good":
                continue

            if os.path.isdir(subfolder_path):
                for img_name in os.listdir(subfolder_path):
                    img_path = os.path.join(subfolder_path, img_name)
                    if is_image_file(img_name):
                        image = preprocess_image(img_path)
                        if image is not None:
                            data.append(image)
                            labels.append(1)  # Anomaly images labeled as 1
    else:
        print(f"Path not found: {test_path}")

    return np.array(data), np.array(labels)

# Load the data
data, labels = load_images()
print(f"Total Images Loaded: {len(data)}")

# Train-test split
train_images, val_images, train_labels, val_labels = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

# Save preprocessed data
np.save(os.path.join(PROCESSED_PATH, "train_images.npy"), train_images)
np.save(os.path.join(PROCESSED_PATH, "val_images.npy"), val_images)
np.save(os.path.join(PROCESSED_PATH, "train_labels.npy"), train_labels)
np.save(os.path.join(PROCESSED_PATH, "val_labels.npy"), val_labels)

print("Preprocessing complete. Data saved successfully.")


Total Images Loaded: 377
Preprocessing complete. Data saved successfully.


In [77]:
train_images.shape, val_images.shape, train_labels.shape, val_labels.shape

((301, 128, 128, 3), (76, 128, 128, 3), (301,), (76,))

In [78]:
print(f"Total Good Images (Label 0): {np.sum(train_labels == 0)}")
print(f"Total Anomaly Images (Label 1): {np.sum(train_labels == 1)}")
print(f"Total Good Images (Label 0): {np.sum(val_labels == 0)}")
print(f"Total Anomaly Images (Label 1): {np.sum(val_labels == 1)}")

Total Good Images (Label 0): 164
Total Anomaly Images (Label 1): 137
Total Good Images (Label 0): 45
Total Anomaly Images (Label 1): 31


In [79]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_classifier():
    """Builds a CNN model for binary classification."""
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build and train the model
model = build_classifier()
model.summary()

d:\CNN Project\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [80]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(train_images, train_labels, validation_data=(val_images, val_labels),
                    epochs=100, batch_size=16, verbose=1,callbacks=[early_stopping])

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 7s 221ms/step - accuracy: 0.5312 - loss: 0.7259 - val_accuracy: 0.5921 - val_loss: 0.6639
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 200ms/step - accuracy: 0.6023 - loss: 0.6848 - val_accuracy: 0.6053 - val_loss: 0.6068
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 201ms/step - accuracy: 0.6786 - loss: 0.6329 - val_accuracy: 0.8684 - val_loss: 0.5054
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 246ms/step - accuracy: 0.8595 - loss: 0.4195 - val_accuracy: 0.9474 - val_loss: 0.2762
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 192ms/step - accuracy: 0.9376 - loss: 0.2141 - val_accuracy: 0.9474 - val_loss: 0.1882
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 185ms/step - accuracy: 0.9317 - loss: 0.2114 - val_accuracy: 0.9474 - val_loss: 0.1541
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 208ms/step - accuracy: 0.9626 - loss: 0.1426 - val_accuracy: 0.9474 - val_loss: 0.1677
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 190ms/step - accuracy: 0.9593 - loss: 0.1280 - val_accu

In [4]:
model.save("my_model_bottle_final.keras")

NameError: name 'model' is not defined

In [3]:
model = tf.keras.models.load_model("my_model_bottle_final.keras")

NameError: name 'tf' is not defined

In [99]:
loss, acc = model.evaluate(val_images, val_labels, verbose=1)
print(f"Validation Accuracy: {acc:.4f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 1.0000 - loss: 0.0037   
Validation Accuracy: 1.0000


In [1]:
def predict_defect(image_path, model):
    image = preprocess_image(image_path)
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    prediction = model.predict(image)[0][0]
    
    if prediction > 0.5:
        print("🔴 Defective Image Detected")
    else:
        print("🟢 Good Image")


In [2]:
image = preprocess_image("D:/CNN Project/mvtec_anomaly_detection/bottle/tes/broken_large/016.png")

NameError: name 'preprocess_image' is not defined

In [114]:
predict_defect("D:/CNN Project/mvtec_anomaly_detection/bottle/test/good/003.png", model)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
🟢 Good Image


In [103]:
predict_defect("D:/CNN Project/mvtec_anomaly_detection/bottle/200.png", model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
🔴 Defective Image Detected
